# Import Packages

In [1]:
import duckdb
import plotly.express as px
import pandas as pd

# Extract Data

Clone repo

In [13]:
!cd .. && cd data && dolt clone post-no-preference/stocks

cloning https://doltremoteapi.dolthub.com/post-no-preference/stocks
data repository already exists: stocks


In [12]:
!cd .. && cd data && dolt clone post-no-preference/earnings

cloning https://doltremoteapi.dolthub.com/post-no-preference/earnings
data repository already exists: earnings


Pull data and merge

In [14]:
!cd .. && cd data && cd stocks && dolt pull

- Pulling.- Pulling.\ Pulling.\ Pulling.| Pulling.| Pulling./ Pulling./ Pulling.- Pulling.- Pulling.\ Pulling.\ Pulling.| Pulling.| Pulling./ Pulling./ Pulling.- Pulling.- Pulling.\ Pulling.\ Pulling.| Pulling.| Pulling./ Pulling./ Pulling.- Pulling.- Pulling.\ Pulling.\ Pulling.| Pulling.| Pulling./ Pulling./ Pulling.- Pulling.- Pulling.\ Pulling.\ Pulling.| Pulling.| Pulling...Updating kg1obkncp8scg90svop415q9nn2n16h0..kg1obkncp8scg90svop415q9nn2n16h0
Everything up-to-date
          

In [15]:
!cd .. && cd data && cd earnings && dolt pull

- Pulling.- Pulling.\ Pulling.\ Pulling.| Pulling.| Pulling./ Pulling./ Pulling.- Pulling.- Pulling.\ Pulling.\ Pulling.| Pulling.| Pulling./ Pulling./ Pulling.- Pulling.- Pulling.\ Pulling.\ Pulling.| Pulling.| Pulling./ Pulling./ Pulling.- Pulling.- Pulling.\ Pulling.\ Pulling.| Pulling.| Pulling./ Pulling./ Pulling.- Pulling.- Pulling...Updating kmeqf73r6gj7g0k6bb5u9rv2ef9a3pvu..kmeqf73r6gj7g0k6bb5u9rv2ef9a3pvu
Everything up-to-date
          

# Create Local Database

In [2]:
# Create connection to local duckdb database
con = duckdb.connect('../db/duck.db')

In [16]:
# disconnect from the database
# con.close()

# Load Data to Database

Earnings

In [18]:
# RUN dolt sql-server in the ../data/earnings directory to start the mysql server 

earnings_tables = [
    "balance_sheet_assets", 
    "balance_sheet_equity", 
    "balance_sheet_liabilities", 
    "cash_flow_statement", 
    "earnings_calendar", 
    "eps_estimate", 
    "eps_history", 
    "income_statement", 
    "rank_score", 
    "sales_estimate"
]

con.sql("ATTACH 'host=localhost user=root port=3306 database=earnings' AS mysql_db_earnings (TYPE mysql_scanner, READ_ONLY);")

for table in earnings_tables:
    con.execute(f"CREATE TABLE {table} AS SELECT * FROM mysql_db_earnings.{table}")

Stocks

In [29]:
# RUN dolt sql-server in the ../data/stocks directory to start the mysql server

stocks_tables = ["dividend", "split", "symbol", "ohlcv"]

con.sql("ATTACH 'host=localhost user=root port=3306 database=stocks' AS mysql_db_stocks (TYPE mysql_scanner, READ_ONLY);")

for table in stocks_tables:
    con.execute(f"CREATE TABLE {table} AS SELECT * FROM mysql_db_stocks.{table}")

# Detach Database

In [33]:
con.sql("SHOW DATABASES;")

┌───────────────────┐
│   database_name   │
│      varchar      │
├───────────────────┤
│ duck              │
│ mysql_db_earnings │
│ mysql_db_stocks   │
└───────────────────┘

In [34]:
con.sql("DETACH mysql_db_stocks;")
con.sql("DETACH mysql_db_earnings;")

In [3]:
con.sql("SHOW DATABASES;")

┌───────────────┐
│ database_name │
│    varchar    │
├───────────────┤
│ duck          │
└───────────────┘

In [4]:
con.sql("SHOW TABLES;")

┌───────────────────────────┐
│           name            │
│          varchar          │
├───────────────────────────┤
│ adj                       │
│ balance_sheet_assets      │
│ balance_sheet_equity      │
│ balance_sheet_liabilities │
│ cash_flow_statement       │
│ div_adj                   │
│ dividend                  │
│ earnings_calendar         │
│ eps_estimate              │
│ eps_history               │
│ income_statement          │
│ ohlcv                     │
│ pricing                   │
│ rank_score                │
│ sales_estimate            │
│ split                     │
│ split_adj                 │
│ symbol                    │
├───────────────────────────┤
│          18 rows          │
└───────────────────────────┘

# IGNORE BELOW

# Transform Data: Create Pricing Tables

### Adjustment Factor: Splits

In [5]:
con.sql("""
CREATE OR REPLACE TABLE split_adj AS
    -- Get the unique symbols from the split table
    WITH base_table AS (
        SELECT DISTINCT act_symbol FROM split
    ),
    -- Create a new table with a row for each symbol and a NULL split for date 2000-01-01
    new_split_table AS (
        SELECT 
            act_symbol,
            DATE '2000-01-01' as ex_date,
            NULL as to_factor,
            NULL as for_factor,
        FROM base_table
        UNION ALL
        SELECT * FROM split
    ),
    -- Calculate the adjustment factor
    split_factors AS (
        -- Calculate adjustment factors and order the rows
        SELECT
            act_symbol,
            ex_date AS adj_date,
            for_factor / to_factor AS adj_factor
        FROM new_split_table
    ),
    -- Order the factors and assign end dates
    ordered_factors AS (
        -- Assign end dates and ensure chronological order
        SELECT
            act_symbol,
            adj_date,
            --STRFTIME(LEAD(adj_date) OVER (PARTITION BY act_symbol ORDER BY adj_date ASC) - INTERVAL '1 DAY', '%Y-%m-%d') AS end_adj_date,
            CAST (LEAD(adj_date) OVER (PARTITION BY act_symbol ORDER BY adj_date ASC) - INTERVAL '1 DAY' AS DATE) AS end_adj_date,
            adj_factor,
            ROW_NUMBER() OVER (PARTITION BY act_symbol ORDER BY adj_date DESC) AS reverse_row_num
        FROM split_factors
    ),
    -- Calculate the cumulative adjustment factor
    cumulative_factors AS (
        -- Compute cumulative adjustment factors in reverse order
        SELECT
            act_symbol,
            adj_date,
            end_adj_date,
            adj_factor,
            -- Reverse chronological cumulative multiplication
            CASE 
                WHEN end_adj_date IS NULL 
                    THEN 1
                WHEN lead(end_adj_date) OVER (PARTITION BY act_symbol ORDER BY adj_date ASC) IS NULL
                    THEN lead(adj_factor) OVER (PARTITION BY act_symbol ORDER BY adj_date ASC)
                ELSE product(adj_factor) OVER (PARTITION BY act_symbol ORDER BY reverse_row_num ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)
                --ELSE lead(adj_factor, 2) OVER (PARTITION BY act_symbol ORDER BY adj_date ASC)  * lead(adj_factor) OVER (PARTITION BY act_symbol ORDER BY adj_date ASC)
            END AS cum_adj_factor
        FROM ordered_factors
    )
    -- Final output
    SELECT
        act_symbol,
        adj_date,
        end_adj_date,
        adj_factor,
        cum_adj_factor
    FROM cumulative_factors
    ORDER BY adj_date ASC;
""")

### Adjustment Factor: Dividends

In [6]:
con.sql("""
SELECT 
    a.*,
    a.amount * COALESCE(b.cum_adj_factor, 1) as amount_adj,
    b.adj_date,
    b.end_adj_date,
    b.adj_factor,
    b.cum_adj_factor
FROM dividend a
LEFT JOIN split_adj b
on a.act_symbol=b.act_symbol AND a.ex_date >= b.adj_date AND (a.ex_date <= b.end_adj_date OR b.end_adj_date IS NULL)
where a.act_symbol='FBL'
LIMIT 10;
""")

┌────────────┬────────────┬───────────────┬────────────┬────────────┬──────────────┬────────────┬────────────────┐
│ act_symbol │  ex_date   │    amount     │ amount_adj │  adj_date  │ end_adj_date │ adj_factor │ cum_adj_factor │
│  varchar   │    date    │ decimal(10,5) │   double   │    date    │     date     │   double   │     double     │
├────────────┼────────────┼───────────────┼────────────┼────────────┼──────────────┼────────────┼────────────────┤
│ FBL        │ 2023-12-27 │      40.02199 │   8.004398 │ 2000-01-01 │ 2024-03-12   │       NULL │            0.2 │
└────────────┴────────────┴───────────────┴────────────┴────────────┴──────────────┴────────────┴────────────────┘

In [115]:
# LATEST
# NEED TO FIRST MULTIPLY DIVIDENDS BY SPLITS ADJUSTMENT
# HAVE ONLY DONE OHLCV ADJUSTMENT WITH SPLITS IN THE FIRST CTE
con.sql("""
CREATE OR REPLACE TABLE div_adj AS (
    WITH ohlcv_with_splits AS (
        select 
            a.date,
            a.act_symbol,
            a.open * COALESCE(f.cum_adj_factor, 1) as open,
            a.high * COALESCE(f.cum_adj_factor, 1) as high,
            a.low * COALESCE(f.cum_adj_factor, 1) as low,
            a.close * COALESCE(f.cum_adj_factor, 1) as close,
            a.volume / COALESCE(f.cum_adj_factor, 1) as volume,
            1 as adj,
            COALESCE(f.cum_adj_factor, 1) as cum_adj_factor,
        FROM ohlcv a
        LEFT JOIN split_adj f on a.act_symbol=f.act_symbol AND a.date >= f.adj_date AND (a.date <= f.end_adj_date OR f.end_adj_date IS NULL)
        LEFT JOIN ohlcv d on a.act_symbol = d.act_symbol and a.date = d.date
    ),
    dividend_with_splits AS (
        SELECT 
            a.act_symbol,
            a.ex_date,
            a.amount * COALESCE(b.cum_adj_factor, 1) as amount,
            'dividend' as type
        FROM dividend a
        LEFT JOIN split_adj b
            on a.act_symbol=b.act_symbol AND a.ex_date >= b.adj_date AND (a.ex_date <= b.end_adj_date OR b.end_adj_date IS NULL)
    ),
    new_div_table AS (
        SELECT 
            a.act_symbol,
            b.ex_date,
            --a.close - b.amount / a.close as adj, 
            (LAG(a.close) OVER (PARTITION BY a.act_symbol ORDER BY a.date ASC) - b.amount) / LAG(a.close) OVER (PARTITION BY a.act_symbol ORDER BY a.date ASC) as adj, 
            'dividend' as type
        FROM ohlcv_with_splits a
        LEFT JOIN dividend_with_splits b 
            on a.act_symbol = b.act_symbol
            and a.date = b.ex_date
    )
        SELECT * FROM new_div_table
        where adj is not null
)
""")

In [7]:
#con.sql("""select * from div_adj a where a.act_symbol = 'TRVG';""")
con.sql("""select * from div_adj a where a.act_symbol = 'FBL';""")

┌────────────┬────────────┬────────────────────┬──────────┐
│ act_symbol │  ex_date   │        adj         │   type   │
│  varchar   │    date    │       double       │ varchar  │
├────────────┼────────────┼────────────────────┼──────────┤
│ FBL        │ 2023-12-27 │ 0.6622901864821534 │ dividend │
└────────────┴────────────┴────────────────────┴──────────┘

In [15]:
con.sql("""select * from split_adj a where a.act_symbol = 'TRVG';""")


┌────────────┬────────────┬──────────────┬────────────┬────────────────┐
│ act_symbol │  adj_date  │ end_adj_date │ adj_factor │ cum_adj_factor │
│  varchar   │    date    │     date     │   double   │     double     │
├────────────┼────────────┼──────────────┼────────────┼────────────────┤
│ TRVG       │ 2000-01-01 │ 2023-11-16   │       NULL │            5.0 │
│ TRVG       │ 2023-11-17 │ NULL         │        5.0 │            1.0 │
└────────────┴────────────┴──────────────┴────────────┴────────────────┘

In [12]:
con.sql("""select * from split a where a.act_symbol = 'TRVG';""")

┌────────────┬────────────┬───────────────┬───────────────┐
│ act_symbol │  ex_date   │   to_factor   │  for_factor   │
│  varchar   │    date    │ decimal(10,5) │ decimal(10,5) │
├────────────┼────────────┼───────────────┼───────────────┤
│ TRVG       │ 2023-11-17 │       1.00000 │       5.00000 │
└────────────┴────────────┴───────────────┴───────────────┘

### Adjustment Factor: Splits & Dividends

In [11]:
con.sql("""
CREATE OR REPLACE TABLE adj AS (
WITH base_table AS (
  -- Get all securities with splits and dividends
  SELECT DISTINCT act_symbol FROM split
  UNION
  SELECT DISTINCT act_symbol FROM dividend
),
empty_adj_table AS (
  -- Create first row so all securities adjustments have a start date
  SELECT 
    act_symbol,
    DATE '2000-01-01' as ex_date,
    NULL as adj,
    NULL as type
  FROM base_table
),
new_div_table AS (
  SELECT 
    b.act_symbol,
    b.ex_date,
    --(c.close - b.amount)/c.close as adj, 
    (LAG(c.close) OVER (PARTITION BY c.act_symbol ORDER BY c.date ASC) - b.amount) / 
      LAG(c.close) OVER (PARTITION BY c.act_symbol ORDER BY c.date ASC) as adj, 
    'dividend' as type
  FROM dividend b 
  RIGHT JOIN ohlcv c 
    on b.act_symbol = c.act_symbol
    and b.ex_date = c.date
),
new_adj_table AS (
  -- Put all the splits and dividends in one nice chronological order
  SELECT * FROM empty_adj_table
  UNION ALL
  SELECT
    a.act_symbol, 
    a.ex_date,
    a.for_factor / a.to_factor as adj,
    'split' as type
  FROM split a
  UNION ALL
  SELECT * FROM div_adj
),
add_end_date AS (
  -- Add the adjustment end date for the adjustment factor 
  -- e.g. split was valid from DATE1 to DATE2, then another event happened on DATE2 + 1 DAY
  SELECT
    *,
    CAST (LEAD(ex_date) OVER (PARTITION BY act_symbol ORDER BY ex_date ASC) - INTERVAL '1 DAY' AS DATE) AS end_adj_date,
  FROM new_adj_table
),
ordered_data AS (
    -- Ensure the data is ordered chronologically
    SELECT
        act_symbol,
        ex_date as adj_date,
        end_adj_date,
        type,
        adj,
        ROW_NUMBER() OVER (PARTITION BY act_symbol ORDER BY adj_date DESC) AS reverse_row_num
    FROM add_end_date
),
cumulative_product AS (
    SELECT
        act_symbol,
        adj_date,
        end_adj_date,
        type,
        adj,
        --reverse_row_num,
        CASE 
          WHEN end_adj_date IS NULL THEN 1
          WHEN lead(end_adj_date) OVER (PARTITION BY act_symbol ORDER BY adj_date ASC) IS NULL THEN lead(adj) OVER (PARTITION BY act_symbol ORDER BY adj_date ASC)
          ELSE product(adj) OVER (PARTITION BY act_symbol ORDER BY reverse_row_num ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)
        END AS cum_adj_factor
    FROM ordered_data
)
SELECT
    *
FROM cumulative_product a
ORDER BY a.act_symbol, adj_date ASC
);
""")

### Pricing

In [25]:
con.sql("""
CREATE OR REPLACE TABLE pricing AS (
    with ohlcv_raw as (
        select 
            *,
            0 as adj,
            1 as cum_adj_factor
        from ohlcv
    ),
    ohlcv_split as (
        select 
            a.date,
            a.act_symbol,
            a.open * COALESCE(f.cum_adj_factor, 1) as open,
            a.high * COALESCE(f.cum_adj_factor, 1) as high,
            a.low * COALESCE(f.cum_adj_factor, 1) as low,
            a.close * COALESCE(f.cum_adj_factor, 1) as close,
            a.volume / COALESCE(f.cum_adj_factor, 1) as volume,
            1 as adj,
            COALESCE(f.cum_adj_factor, 1) as cum_adj_factor,
        FROM ohlcv a
        LEFT JOIN split_adj f on a.act_symbol=f.act_symbol AND a.date >= f.adj_date AND (a.date <= f.end_adj_date OR f.end_adj_date IS NULL)
        LEFT JOIN ohlcv d on a.act_symbol = d.act_symbol and a.date = d.date
    ),
    ohlcv_adj as (
        SELECT 
            a.date,
            a.act_symbol,
            a.open * COALESCE(c.cum_adj_factor, 1) as open,
            a.high * COALESCE(c.cum_adj_factor, 1) as high,
            a.low * COALESCE(c.cum_adj_factor, 1) as low,
            a.close * COALESCE(c.cum_adj_factor, 1) as close,
            a.volume / COALESCE(f.cum_adj_factor, 1) as volume,
            2 as adj,
            COALESCE(c.cum_adj_factor, 1) as cum_adj_factor,
        FROM ohlcv a
        LEFT JOIN adj c on a.act_symbol=c.act_symbol AND a.date >= c.adj_date AND (a.date <= c.end_adj_date OR c.end_adj_date IS NULL)
        LEFT JOIN split_adj f on a.act_symbol=f.act_symbol AND a.date >= f.adj_date AND (a.date <= f.end_adj_date OR f.end_adj_date IS NULL)
        LEFT JOIN ohlcv d on a.act_symbol = d.act_symbol and a.date = d.date
    )
    select * from ohlcv_raw
    union
    select * from ohlcv_split
    union
    select * from ohlcv_adj
);
"""
)

### Pricing Extended

In [22]:
con.sql("""
CREATE OR REPLACE TABLE pricing_extended AS (
  SELECT 
        a.date,
        a.act_symbol,
        a.open as open,
        a.open * COALESCE(c.cum_adj_factor, 1) as open_adj,
        a.high as high,
        a.high * COALESCE(c.cum_adj_factor, 1) as high_adj,
        a.low as low,
        a.low * COALESCE(c.cum_adj_factor, 1) as low_adj,
        a.close as close,
        a.close * COALESCE(c.cum_adj_factor, 1) as close_adj,
        a.volume as volume,
        a.volume / COALESCE(f.cum_adj_factor, 1) as volume_adj,
        e.for_factor / e.to_factor as split_ratio,
        b.amount as div,
        c.adj_date,
        c.end_adj_date,
        c.type,
        c.adj,
        COALESCE(c.cum_adj_factor, 1) as cum_adj_factor,
    FROM ohlcv a
    LEFT JOIN split e on a.act_symbol=e.act_symbol and a.date=e.ex_date
    LEFT JOIN dividend b ON a.act_symbol=b.act_symbol AND a.date=b.ex_date
    LEFT JOIN adj c on a.act_symbol=c.act_symbol AND a.date >= c.adj_date AND (a.date <= c.end_adj_date OR c.end_adj_date IS NULL)
    LEFT JOIN ohlcv d on a.act_symbol = d.act_symbol and a.date = d.date
    LEFT JOIN split_adj f on a.act_symbol=f.act_symbol AND a.date >= f.adj_date AND (a.date <= f.end_adj_date OR f.end_adj_date IS NULL)
    order by a.date
);
""")

RuntimeError: Query interrupted

## Test Plot

In [32]:
con.sql("""select * from pricing a where adj=2 and a.act_symbol = 'MBI' order by date desc;""")

┌────────────┬────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────┬───────┬─────────────────────┐
│    date    │ act_symbol │        open        │        high        │        low         │       close        │   volume   │  adj  │   cum_adj_factor    │
│    date    │  varchar   │       double       │       double       │       double       │       double       │   double   │ int32 │       double        │
├────────────┼────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────┼───────┼─────────────────────┤
│ 2025-01-16 │ MBI        │               6.34 │               6.44 │                6.3 │               6.37 │   252251.0 │     2 │                 1.0 │
│ 2025-01-15 │ MBI        │               6.29 │               6.47 │               6.25 │               6.36 │   188010.0 │     2 │                 1.0 │
│ 2025-01-14 │ MBI        │               6.35 │               6.41 │ 

In [34]:
# Step 1: Query the DuckDB table into a Pandas DataFrame
query = """
SELECT 
    *
    --, log(close) as log_close
from pricing a 
where a.act_symbol ='MBI' and a.adj = 1
order by date asc
"""
df = con.execute(query).fetchdf()
df.tail()

# Step 2: Plot the data using Plotly Express
fig = px.line(df, x='date', y='close')
fig.show()

# Testing Issues: Close Adj < 0

In [123]:
con.sql("""select a.date, a.act_symbol, a.close, a.cum_adj_factor from pricing a where adj=1 and a.act_symbol='TRVG' 
AND a.date > '2023-11-10' order by date asc;""")

┌────────────┬────────────┬────────────────────┬────────────────┐
│    date    │ act_symbol │       close        │ cum_adj_factor │
│    date    │  varchar   │       double       │     double     │
├────────────┼────────────┼────────────────────┼────────────────┤
│ 2023-11-13 │ TRVG       │ 5.3500000000000005 │            5.0 │
│ 2023-11-14 │ TRVG       │                3.2 │            5.0 │
│ 2023-11-15 │ TRVG       │               3.25 │            5.0 │
│ 2023-11-16 │ TRVG       │                3.0 │            5.0 │
│ 2023-11-17 │ TRVG       │               2.81 │            1.0 │
│ 2023-11-20 │ TRVG       │                2.8 │            1.0 │
│ 2023-11-21 │ TRVG       │               2.69 │            1.0 │
│ 2023-11-22 │ TRVG       │               2.67 │            1.0 │
│ 2023-11-24 │ TRVG       │               2.74 │            1.0 │
│ 2023-11-27 │ TRVG       │               2.75 │            1.0 │
│     ·      │  ·         │                 ·  │             ·  │
│     ·   

In [87]:
con.sql("select * from adj a where a.act_symbol = 'FBL';")

┌────────────┬────────────┬──────────────┬──────────┬────────────────────┬────────────────────┐
│ act_symbol │  adj_date  │ end_adj_date │   type   │        adj         │   cum_adj_factor   │
│  varchar   │    date    │     date     │ varchar  │       double       │       double       │
├────────────┼────────────┼──────────────┼──────────┼────────────────────┼────────────────────┤
│ FBL        │ 2000-01-01 │ 2023-12-26   │ NULL     │               NULL │ 0.1324580372964307 │
│ FBL        │ 2023-12-27 │ 2024-03-12   │ dividend │ 0.6622901864821534 │                0.2 │
│ FBL        │ 2024-03-13 │ NULL         │ split    │                0.2 │                1.0 │
└────────────┴────────────┴──────────────┴──────────┴────────────────────┴────────────────────┘

In [118]:
con.sql("SELECT * FROM ohlcv a where a.act_symbol = 'TRVG' AND date > '2023-11-01'")

┌────────────┬────────────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────┐
│    date    │ act_symbol │     open     │     high     │     low      │    close     │ volume  │
│    date    │  varchar   │ decimal(7,2) │ decimal(7,2) │ decimal(7,2) │ decimal(7,2) │  int64  │
├────────────┼────────────┼──────────────┼──────────────┼──────────────┼──────────────┼─────────┤
│ 2023-11-02 │ TRVG       │         1.04 │         1.13 │         1.02 │         1.04 │  916074 │
│ 2023-11-03 │ TRVG       │         1.06 │         1.10 │         1.01 │         1.07 │  468723 │
│ 2023-11-06 │ TRVG       │         1.03 │         1.09 │         0.99 │         1.07 │  385206 │
│ 2023-11-07 │ TRVG       │         1.07 │         1.08 │         1.01 │         1.05 │  259627 │
│ 2023-11-08 │ TRVG       │         1.06 │         1.06 │         1.00 │         1.03 │  455825 │
│ 2023-11-09 │ TRVG       │         1.04 │         1.05 │         1.00 │         1.02 │  384357 │
│ 2023-11-10 │ TRVG 

In [88]:
con.sql("SELECT * FROM dividend a where a.act_symbol = 'FBL'")

┌────────────┬────────────┬───────────────┐
│ act_symbol │  ex_date   │    amount     │
│  varchar   │    date    │ decimal(10,5) │
├────────────┼────────────┼───────────────┤
│ FBL        │ 2023-12-27 │      40.02199 │
└────────────┴────────────┴───────────────┘

In [90]:
con.sql("SELECT * FROM split a where a.act_symbol = 'FBL'")

┌────────────┬────────────┬───────────────┬───────────────┐
│ act_symbol │  ex_date   │   to_factor   │  for_factor   │
│  varchar   │    date    │ decimal(10,5) │ decimal(10,5) │
├────────────┼────────────┼───────────────┼───────────────┤
│ FBL        │ 2024-03-13 │       5.00000 │       1.00000 │
└────────────┴────────────┴───────────────┴───────────────┘

In [89]:
con.sql("SELECT * FROM adj a where a.act_symbol = 'FBL'")

┌────────────┬────────────┬──────────────┬──────────┬────────────────────┬────────────────────┐
│ act_symbol │  adj_date  │ end_adj_date │   type   │        adj         │   cum_adj_factor   │
│  varchar   │    date    │     date     │ varchar  │       double       │       double       │
├────────────┼────────────┼──────────────┼──────────┼────────────────────┼────────────────────┤
│ FBL        │ 2000-01-01 │ 2023-12-26   │ NULL     │               NULL │ 0.1324580372964307 │
│ FBL        │ 2023-12-27 │ 2024-03-12   │ dividend │ 0.6622901864821534 │                0.2 │
│ FBL        │ 2024-03-13 │ NULL         │ split    │                0.2 │                1.0 │
└────────────┴────────────┴──────────────┴──────────┴────────────────────┴────────────────────┘

In [13]:
con.sql("""select a.date, a.act_symbol, a.close, a.cum_adj_factor from pricing a where adj=2 and a.act_symbol='TRVG' and date > '2023-11-05' order by date asc;""")

┌────────────┬────────────┬────────────────────┬────────────────────┐
│    date    │ act_symbol │       close        │   cum_adj_factor   │
│    date    │  varchar   │       double       │       double       │
├────────────┼────────────┼────────────────────┼────────────────────┤
│ 2023-11-06 │ TRVG       │             -8.775 │ -8.200934579439252 │
│ 2023-11-07 │ TRVG       │ -8.610981308411215 │ -8.200934579439252 │
│ 2023-11-08 │ TRVG       │  -8.44696261682243 │ -8.200934579439252 │
│ 2023-11-09 │ TRVG       │ -8.364953271028037 │ -8.200934579439252 │
│ 2023-11-10 │ TRVG       │  -8.44696261682243 │ -8.200934579439252 │
│ 2023-11-13 │ TRVG       │             -8.775 │ -8.200934579439252 │
│ 2023-11-14 │ TRVG       │                3.2 │                5.0 │
│ 2023-11-15 │ TRVG       │               3.25 │                5.0 │
│ 2023-11-16 │ TRVG       │                3.0 │                5.0 │
│ 2023-11-17 │ TRVG       │               2.81 │                1.0 │
│     ·      │  ·   